In [1]:
# Import modules
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils.np_utils import to_categorical
from keras.layers import Input
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras import layers
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping
print('Notebook run using keras:', keras.__version__)

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

import cv2
import os
import shutil
import itertools
import imutils

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_curve, auc
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from skimage.io import imread
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools

import math
from PIL import Image
from scipy import misc
from os import listdir
from os.path import isfile, join
from scipy import misc
from random import shuffle
from collections import Counter
from numpy import expand_dims
import tensorflow as tf

init_notebook_mode(connected=True)
RANDOM_SEED = 123
IMG_SIZE = (224,224)

Notebook run using keras: 2.12.0


In [2]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Develop VGG-16 Model Architecture
model = Sequential()

# Block 1
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 2
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 3
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 4
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 5
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Fully connected layers
model.add(layers.Dropout(0.3))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=0.0003, beta_1=0.9, beta_2=0.999, decay=0.0, amsgrad=False),
              metrics=["accuracy"])

In [4]:
# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.SGD(learning_rate=0.001, momentum=0.9),
              metrics=['accuracy'])

In [5]:
# Function to load data
def load_data(dir_path):
    """
    Load resized images as np.arrays to workspace
    """
    X = []
    y = []
    i = 0
    labels = dict()
    for path in tqdm(sorted(os.listdir(dir_path))):
        if not path.startswith('.'):
            labels[i] = path
            for file in os.listdir(dir_path + path):
                if not file.startswith('.'):
                    img = cv2.imread(dir_path + path + '/' + file)
                    X.append(img)
                    y.append(i)
            i += 1
    X = np.array(X)
    y = np.array(y)
    print(f'{len(X)} images loaded from {dir_path} directory.')
    return X, y, labels

In [6]:
# Load images into set variables
X_train_crop, y_train, labels = load_data('/content/drive/MyDrive/Parkinsons-DaTscan-master/Implementation/TRAIN_CROP/')
X_test_crop, y_test, _ = load_data('/content/drive/MyDrive/Parkinsons-DaTscan-master/Implementation/TEST_CROP/')
X_val_crop, y_val, _ = load_data('/content/drive/MyDrive/Parkinsons-DaTscan-master/Implementation/VAL_CROP/')

100%|██████████| 2/2 [00:08<00:00,  4.38s/it]
<ipython-input-5-b4b0d15ef2bf>:19: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



522 images loaded from /content/drive/MyDrive/Parkinsons-DaTscan-master/Implementation/TRAIN_CROP/ directory.


100%|██████████| 2/2 [00:23<00:00, 11.83s/it]


65 images loaded from /content/drive/MyDrive/Parkinsons-DaTscan-master/Implementation/TEST_CROP/ directory.


100%|██████████| 2/2 [00:27<00:00, 13.66s/it]

74 images loaded from /content/drive/MyDrive/Parkinsons-DaTscan-master/Implementation/VAL_CROP/ directory.


In [7]:
# Resize images to fit input layer of CNN ie 224x224
def preprocess_imgs(set_name, img_size):
    """
    Resize and apply VGG-19 preprocessing
    """
    set_new = []
    for img in set_name:
        img = cv2.resize(
            img,
            dsize=img_size,
            interpolation=cv2.INTER_CUBIC
        )
        set_new.append(preprocess_input(img))
    return np.array(set_new)

In [8]:
# Load resized images into final set variables
X_train_prep = preprocess_imgs(set_name=X_train_crop, img_size=IMG_SIZE)
X_test_prep = preprocess_imgs(set_name=X_test_crop, img_size=IMG_SIZE)
X_val_prep = preprocess_imgs(set_name=X_val_crop, img_size=IMG_SIZE)

In [ ]:
# Train the model
hist = model.fit(
    steps_per_epoch=50,
    x = X_train_prep,
    y = y_train,  
    validation_data= (X_val_prep, y_val), 
    validation_steps=10,
    epochs=30,
)

Epoch 1/30
50/50 [==============================] - ETA: 0s - loss: 0.5645 - accuracy: 0.7261 

In [ ]:
# Validate on val set
predictions = model.predict(X_val_prep)
predictions = [1 if x>0.5 else 0 for x in predictions]

accuracy = accuracy_score(y_val, predictions)
print('Val Accuracy = %.2f' % accuracy)

In [ ]:
# Validate on test set
predictions = model.predict(X_test_prep)
predictions = [1 if x>0.5 else 0 for x in predictions]

accuracy = accuracy_score(y_test, predictions)
print('Test Accuracy = %.2f' % accuracy)

In [ ]:
# Plot the accuracy and loss over time
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
loss = hist.history['loss']
val_loss = hist.history['val_loss']
epochs_range = range(1, len(hist.epoch) + 1)


plt.figure(figsize=(10,5))
plt.plot(epochs_range, acc, label='Training')
plt.plot(epochs_range, val_acc, label='Validation')
plt.legend(loc="best")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.grid(which='major', color='#666666', linestyle='-')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(epochs_range, loss, label='Training')
plt.plot(epochs_range, val_loss, label='Validation')
plt.legend(loc="best")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.grid(which='major', color='#666666', linestyle='-')
plt.tight_layout()
plt.show()

In [ ]:
# Print confusion matrix
import seaborn as sn

confusion_mtx = confusion_matrix(y_test, predictions)
ax = plt.axes()
sn.heatmap(confusion_mtx, annot=True,annot_kws={"size": 25}, cmap="Blues", ax = ax)
ax.set_title('Test Accuracy', size=14)
plt.show()

In [ ]:
print("Classification Report:")
report = classification_report(y_test, predictions, target_names=["Yes", "No"])
print(report)

In [ ]:
# Calculate ROC curve and AUC
fpr, tpr, _ = roc_curve(y_test, predictions)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()